In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML
from IPython.lib.pretty import pprint
from itertools import chain

In [ ]:
from mockdown.view import View

In [4]:
sample_view = View("p", (0, 0, 100, 100), children=[
    View("a1", (10, 10, 45, 90), children=[
        View("b11", (20, 20, 35, 45)),
        View("b12", (20, 55, 35, 80))
    ]),
    View("a2", (55, 10, 90, 90), children=[
        View("b21", (65, 20, 80, 45)),
        View("b22", (65, 55, 80, 80))
    ])
])

In [5]:
html_str = str(sample_view.to_display_html(scale=2))
display(HTML(html_str))

In [6]:
from mockdown.visibility import visible_pairs

edge_pairs = visible_pairs(sample_view, deep=True)
anchor_pairs = [(e1.anchor, e2.anchor) for (e1, e2) in edge_pairs]

In [7]:
html_str = str(sample_view.to_display_html(edge_pairs, scale=2))
display(HTML(html_str))

In [8]:
from mockdown.logic import constraint_pairs
answer = constraint_pairs(sample_view, anchor_pairs)

for spacing in answer:
    print(f"{spacing['V']}.{spacing['A']} to {spacing['W']}.{spacing['B']}")

a1.right to a2.left
b11.bottom to b12.top
b21.bottom to b22.top
p.top to a1.top
p.left to a1.left
p.bottom to a1.bottom
p.top to a2.top
p.bottom to a2.bottom
p.right to a2.right
a1.top to b11.top
a1.left to b11.left
a1.right to b11.right
a1.left to b12.left
a1.bottom to b12.bottom
a1.right to b12.right
a2.top to b21.top
a2.left to b21.left
a2.right to b21.right
a2.left to b22.left
a2.bottom to b22.bottom
a2.right to b22.right
